In [1]:
import time
import numpy as np
import mujoco, mujoco_viewer
import mediapy as media

from scipy.optimize import minimize

from policy_infer import RealtimeController
from utils import quaternion_to_euler_array
from configs import env_cfg, cfg

In [2]:
def load_model_and_data(model_path):
    """
    加载MuJoCo模型和数据
    """
    model = mujoco.MjModel.from_xml_path(model_path)
    data = mujoco.MjData(model)
    return model, data

model, data = load_model_and_data(cfg.sim_config.mujoco_model_path)
viewer = mujoco_viewer.MujocoViewer(model, data)

In [3]:
def get_object_properties(model, data, object_name):
    """
    获取物体的位置和尺寸
    """
    body_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, object_name)
    if body_id == -1:
        raise ValueError(f"Object '{object_name}' not found in the model.")
    
    # 获取物体的位置
    pos = data.xpos[body_id].copy()
    
    # 获取物体的几何体尺寸
    # 获取物体所有的geom ID
    geom_ids = []
    for geom_id in range(model.ngeom):
        if model.geom_bodyid[geom_id] == body_id:
            geom_ids.append(geom_id)
    
    if not geom_ids:
        raise ValueError(f"No geom found for object '{object_name}'.")
    
    # 假设第一个geom是主要的几何体
    size = model.geom_size[geom_ids[0]].copy()
    
    return pos, size

pos, size = get_object_properties(model, data, "left_wrist_roll_link")
print(pos, size)

[0. 0. 0.] [0.03150297 0.04033012 0.06490997]


In [4]:
def compute_hand_targets(object_pos, object_size, hand_offset=0.05):
    """
    计算左右手的目标位置
    """
    # 假设物体在y轴上分布，左右手分别在物体的左右两侧
    left_target = object_pos.copy()
    right_target = object_pos.copy()
    
    left_target[1] += (object_size[1]/2 + hand_offset)
    right_target[1] -= (object_size[1]/2 + hand_offset)
    
    return left_target, right_target

left_target, right_target = compute_hand_targets([0.3, 0, 1.2], [0.1, 0.1, 0.1])
print(left_target, right_target)

[0.3, 0.1, 1.2] [0.3, -0.1, 1.2]


In [5]:
def inverse_kinematics(model, data, target_pos, joint_names, eef_name):
    """
    简单的逆运动学实现，使用优化的方法最小化末端执行器与目标位置的距离
    """
    # 获取臂的关节索引
    joint_ids = []
    for j in range(model.njnt):
        joint_name = mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_JOINT, j)
        if joint_name and joint_name in joint_names:
            joint_ids.append(j)
    
    if not joint_ids:
        raise ValueError(f"No joints found for arm '{eef_name}'.")
    
    # 假设末端执行器的body名称为 "{arm_name}_link"
    eef_name = f"{eef_name}_link"
    eef_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, eef_name)
    if eef_id == -1:
        raise ValueError(f"End-effector '{eef_name}' not found in the model.")
    
    def objective(q):
        # 保存当前关节角度
        original_qpos = data.qpos.copy()
        
        # 设置新的关节角度
        data.qpos[joint_ids] = q
        mujoco.mj_forward(model, data)
        
        # 获取末端执行器位置
        eef_pos = data.xpos[eef_id]
        
        # 恢复原始关节角度
        data.qpos[:] = original_qpos
        
        # 计算目标与末端执行器的距离平方和
        return np.sum((eef_pos - target_pos)**2)
    
    # 初始猜测为当前关节角度
    initial_q = data.qpos[joint_ids].copy()
    
    # 优化
    res = minimize(objective, initial_q, method='BFGS')
    
    if res.success:
        return res.x, joint_ids
    else:
        raise ValueError(f"Inverse kinematics for {eef_name} failed.")

left_joint_names = ['left_shoulder_pitch', 'left_shoulder_roll', 'left_shoulder_yaw', 'left_elbow_pitch', 'left_elbow_yaw', 'left_wrist_pitch', 'left_wrist_roll']
right_joint_names = ['right_shoulder_pitch', 'right_shoulder_roll', 'right_shoulder_yaw', 'right_elbow_pitch', 'right_elbow_yaw', 'right_wrist_pitch', 'right_wrist_roll']

left_target_angles, left_joint_ids = inverse_kinematics(model, data, left_target, left_joint_names, 'left_wrist_roll')
right_target_angles, right_joint_ids = inverse_kinematics(model, data, right_target, right_joint_names, 'right_wrist_roll')

print(f"Left Arm Target Angles: {left_target_angles}")
print(f"Right Arm Target Angles: {right_target_angles}")

Left Arm Target Angles: [0.51404269 0.41324803 0.10687139 1.47568289 0.03427999 0.
 0.        ]
Right Arm Target Angles: [-0.51410687 -0.41346147 -0.10674939 -1.47632274 -0.03458121  0.
  0.        ]


### mj_forward 静态计算

In [6]:
def smooth_joint_movement_simultaneous(model, data, viewer, left_joint_ids, left_target_angles, 
                                       right_joint_ids, right_target_angles, steps=100, delay=0.01):
    """
    同时平滑地移动双臂到目标角度
    """
    # 获取当前左右臂的关节角度
    current_left_angles = data.qpos[left_joint_ids].copy()
    current_right_angles = data.qpos[right_joint_ids].copy()
    
    # 计算每一步需要增加的角度
    delta_left = (left_target_angles - current_left_angles) / steps
    delta_right = (right_target_angles - current_right_angles) / steps
    
    for step in range(steps):
        # 更新左臂和右臂的关节角度
        data.qpos[left_joint_ids] += delta_left
        data.qpos[right_joint_ids] += delta_right
        
        # 计算前向动力学
        mujoco.mj_forward(model, data)
        
        # 渲染视图
        if viewer is not None and viewer.is_alive:
            viewer.render()
        else:
            break
        
        # 延迟以控制移动速度
        time.sleep(delay)
    
    # 确保最终位置准确
    data.qpos[left_joint_ids] = left_target_angles
    data.qpos[right_joint_ids] = right_target_angles
    mujoco.mj_forward(model, data)
    if viewer is not None and viewer.is_alive:
        viewer.render()

# 示例调用
# smooth_joint_movement_simultaneous(
#     model, 
#     data, 
#     viewer, 
#     left_joint_ids, 
#     left_target_angles, 
#     right_joint_ids, 
#     right_target_angles,
#     steps=100,
#     delay=0.01
# )


### mj_step 动态仿真

In [7]:
def get_actuator_ids(model, joint_names):
    """
    根据关节名称获取对应的执行器ID。
    假设每个关节有一个对应的执行器，且执行器名称与关节名称匹配。
    """
    actuator_ids = []
    for name in joint_names:
        actuator_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, f"motor_{name}")
        actuator_ids.append(actuator_id)
        print(f"Actuator ID for joint {name}: {actuator_id}")
    return actuator_ids

# 获取对应的执行器ID
left_actuator_ids = get_actuator_ids(model, left_joint_names)
right_actuator_ids = get_actuator_ids(model, right_joint_names)
actuator_ids = left_actuator_ids + right_actuator_ids

Actuator ID for joint left_shoulder_pitch: 0
Actuator ID for joint left_shoulder_roll: 1
Actuator ID for joint left_shoulder_yaw: 2
Actuator ID for joint left_elbow_pitch: 3
Actuator ID for joint left_elbow_yaw: 4
Actuator ID for joint left_wrist_pitch: 5
Actuator ID for joint left_wrist_roll: 6
Actuator ID for joint right_shoulder_pitch: 7
Actuator ID for joint right_shoulder_roll: 8
Actuator ID for joint right_shoulder_yaw: 9
Actuator ID for joint right_elbow_pitch: 10
Actuator ID for joint right_elbow_yaw: 11
Actuator ID for joint right_wrist_pitch: 12
Actuator ID for joint right_wrist_roll: 13


In [8]:
def pd_control(data, actuator_ids, target_angles, Kp, Kd):
    """
    计算并应用 PD 控制扭矩。

    参数：
    - data: MuJoCo 数据对象
    - actuator_ids: 执行器ID列表
    - target_angles: 目标角度数组
    - Kp: 比例增益数组
    - Kd: 微分增益数组
    """
    # 获取当前关节角度和速度
    current_angles = np.array([data.qpos[mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_JOINT, joint)] for joint in left_joint_names + right_joint_names])
    current_velocities = np.array([data.qvel[mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_JOINT, joint)] for joint in left_joint_names + right_joint_names])

    # 计算误差
    pos_error = target_angles - current_angles
    vel_error = -current_velocities

    # 计算扭矩
    torques = Kp * pos_error + Kd * vel_error

    # 应用扭矩到对应的执行器
    data.ctrl[actuator_ids] = torques

In [9]:
def smooth_joint_movement_pd(model, data, viewer, actuator_ids, target_angles, Kp, Kd, steps=100, delay=0.01):
    """
    使用 PD 控制器平滑地同时移动双臂到目标角度。

    参数：
    - model: MuJoCo 模型
    - data: MuJoCo 数据
    - viewer: 渲染视图对象
    - actuator_ids: 执行器ID列表
    - target_angles: 目标角度数组
    - Kp: 比例增益数组
    - Kd: 微分增益数组
    - steps: 总步数
    - delay: 每步的延迟时间（秒）
    """
    # 线性插值目标角度
    start_angles = np.array([data.qpos[mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_JOINT, joint)] for joint in left_joint_names + right_joint_names])
    delta = (target_angles - start_angles) / steps

    for step in range(steps):
        # 计算当前目标角度
        current_target = start_angles + delta * (step + 1)

        # 应用 PD 控制
        pd_control(data, actuator_ids, current_target, Kp, Kd)

        # 步进仿真
        mujoco.mj_step(model, data)

        # 渲染
        if viewer is not None and viewer.is_alive:
            viewer.render()
        else:
            break

        # 延迟
        time.sleep(delay)

    # 确保最终位置准确
    pd_control(data, actuator_ids, target_angles, Kp, Kd)
    mujoco.mj_step(model, data)
    if viewer is not None and viewer.is_alive:
        viewer.render()

In [10]:
# 设置 PD 增益
Kp = np.array([200, 150, 150, 120, 100, 80, 80, 200, 150, 150, 120, 100, 80, 80])
Kd = np.array([10, 8, 8, 6, 5, 4, 4, 10, 8, 8, 6, 5, 4, 4])

# 定义目标角度（弧度）
target_angles = np.concatenate([left_target_angles, right_target_angles])

# 调用平滑移动函数
smooth_joint_movement_pd(
    model,
    data,
    viewer,
    actuator_ids,
    target_angles,
    Kp,
    Kd,
    steps=200,
    delay=0.01
)

### 持续应用 PD 控制以维持目标姿态

In [ ]:
while viewer.is_alive:
    # 应用 PD 控制以维持目标姿态
    pd_control(data, actuator_ids, target_angles, Kp, Kd)

    # 步进仿真
    mujoco.mj_step(model, data)

    # 渲染
    viewer.render()

    # 可选延迟，根据需要调整
    time.sleep(0.01)